# Evaluate regression model

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay import database

In [ ]:
df_regression = database.load(rolldecay_table_name='rolldecay_quadratic_b_regression', 
                              limit_score=-np.inf)

df_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', 
                              limit_score=-np.inf)


In [ ]:
df_regression.describe()

In [ ]:
fig,ax=plt.subplots()
df_regression['score'].hist(bins=40, ax=ax)

In [ ]:
fig,ax=plt.subplots()
df_ikeda['score'].hist(bins=40, ax=ax)

In [ ]:
df_regression['score'].describe()

In [ ]:
df_ikeda['score'].describe()

In [ ]:
mask=df_regression['ship_speed']==0
df_regression_zero=df_regression.loc[mask].copy()

mask=df_ikeda['ship_speed']==0
df_ikeda_zero=df_ikeda.loc[mask].copy()

In [ ]:
df_regression_zero['score'].describe()

In [ ]:
df_ikeda_zero['score'].describe()

In [ ]:
df_compare = pd.merge(left=df_regression_zero, right=df_ikeda_zero, how='inner',
                      left_index=True, right_index=True, suffixes=('_regression','_ikeda'))
                      

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='score_ikeda',y='score_regression', ax=ax, style='o', alpha=0.5)

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_1_ikeda',y='B_1_regression', ax=ax, style='o', alpha=0.5)

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_2_ikeda',y='B_2_regression', ax=ax, style='o', alpha=0.5)